In [1]:
import pandas as pd

art_df = pd.read_csv("article_df.csv", index_col=0)

In [2]:
# !pip install konlpy

from konlpy.tag import Okt
from tqdm import tqdm

In [3]:
art_df.dropna(inplace=True)

In [4]:
# 각 기사의 본문에서 명사, 형용사, 동사, 부사를 추출하여 리스트에 저장 후 딕셔너리에 저장
okt = Okt()
poss_dic = {}

for i in tqdm(art_df.index):
    ## 1.
    main = art_df.loc[i, "main"]
    poss = okt.pos(main, norm=True, stem=True)
	
    ## 2.
    poss_lst = []
    for word, tag in poss:
        if tag in ['Noun','Adjective','Verb','Adverb']:
            poss_lst.append(word)
    ## 3. 
    poss_dic[i] = poss_lst

100%|██████████| 3299/3299 [01:50<00:00, 29.83it/s]


In [5]:
# 모든 뉴스 기사 본문에서 중복을 없앤 형태소 리스트
all_unique_words = []

for words in poss_dic.values():
    all_unique_words.extend(words)
    all_unique_words = list(set(all_unique_words))

In [7]:
print(all_unique_words)

['전투기', '외교관', '재현', '관례', '극복', '김현주', '자행', '월요일', '기다', '실으', '아스', '군벌', '리병덕', '파블로', '거렸', '이보', '떼', '확약', '형제자매', '스크류바', '부에노스아이레스', '입각', '로티', '바뀌다', '알짜', '보성군', '버스', '내한', '저온', '저작권', '슬리퍼', '박주연', '초청', '라당스', '간식', '뇌경색', '명파', '구독', '과부', '부적절하다', '고신대', '트리', '희소성', '키로프', '이데일리', '양극화', '접점', '홈트', '쇼핑몰', '저격', '공로', '바탕', '중동', '의자', '수선', '현기', '군검', '영어', '폐차', '너희', '김병', '율도', '법대', '거머', '직권남용죄', '붉다', '대표이사', '세제', '대동소이하다', '빌딩', '레퍼토리', '병역의무', '김병주', '적절', '관청', '격인', '령', '폭제', '심위', '국기', '예수', '퍼트', '영감', '유지니', '황기선', '호국', '리카르도', '가격담합', '그립다', '부추기다', '관전', '기본소득', '로테르담', '증오', '숙의', '병상', '학습자', '키스', '깔끔하다', '튀어', '원판', '윤계', '울음소리', '들어서다', '죽림', '수감', '복개', '물러나다', '박지훈', '협회', '증설', '당시', '투명하다', '훈', '가로', '서부', '헤어', '환속', '거처', '제풀이', '픽텟', '창리', '소통', '파워', '양천구', '레귤러', '키르기스스탄', '새', '타도', '떼다', '교육자', '빌런', '정이', '남침', '이면', '불', '쪼개지다', '펄', '타임', '치면', '티베트인', '덥다', '휘청대다', '교양', '조약', '패배', '법조', '가만', '인버터', '베일', '불복', '독약', '신문', 

In [8]:
# 각각의 기사마다 특정 단어가 몇 번 나왔는지를 나타내는 데이터프레임 (BoW)

datas = []
for i in tqdm(art_df.index):
    words = poss_dic[i]  # 형태소가 분리된 단어들을 모아놓은 리스트
    vc = pd.Series(words).value_counts() # (key: 형태소, values: 빈도수)
    data = vc.to_dict()
    datas.append(data)

df = pd.DataFrame(datas, index=art_df.index, columns=all_unique_words)

100%|██████████| 3299/3299 [00:00<00:00, 7067.07it/s]


In [9]:
df = df.fillna(0)

In [10]:
section_n = art_df["section"].astype(str).str[-1:].astype(int)
df["section"] = section_n

In [16]:
df.sum()

전투기           1.0
외교관           7.0
재현            6.0
관례            2.0
극복           76.0
            ...  
숱             1.0
강매           23.0
시선집중          4.0
신의성실          3.0
section    5483.0
Length: 17469, dtype: float64

In [17]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [18]:
# 데이터 로드

X = df.drop("section", axis=1)
Y = df["section"]
x_train, x_test, y_train, y_test = train_test_split(X, Y)

In [19]:
# 모델 학습
model = RandomForestClassifier(n_estimators=300)
model.fit(x_train, y_train)

RandomForestClassifier(n_estimators=300)

In [20]:
# 평가
print("훈련 세트 정확도: {:.3f}".format(model.score(x_train, y_train)))
print("테스트 세트 정확도: {:.3f}".format(model.score(x_test, y_test)))

훈련 세트 정확도: 0.963
테스트 세트 정확도: 0.884


In [21]:
imp = model.feature_importances_
imp_df = pd.DataFrame(imp, index=X.columns, columns=["imp"])
imp_df = imp_df.sort_values("imp", ascending=False)

In [22]:
imp_df

,imp
민주당,0.006094
제품,0.006047
현지,0.005439
기업,0.004998
혐의,0.004711
...,...
발할라,0.000000
오프,0.000000
차종,0.000000
세븐일레븐,0.000000


In [24]:
text = """
원·달러 환율, 3개월 만에 1320원대 재진입…美 물가·韓 금리 동결 영향
외환당국 美 긴축 장기화 우려에 변동성↑…외화자금 모니터링 강화

...


국제 경제·정치 상황의 변화가 우리 경제에 영향을 미칠 것이라며 국내 외환·금융시장 안정성을 
유지하도록 외화자금 유출입 모니터링 등에 긴밀히 공조할 필요가 있다고 말했다.
"""

In [25]:
# text에서 명사, 동사, 형용사, 부사인 형태소만 추출
raw = okt.pos(text, norm=True, stem=True)

words = []
for word, pos in raw:
    if pos in ["Noun", "Verb", "Adjective", "Adverb"]:
        words.append(word)
        
vc = pd.Series(words).value_counts()

In [26]:
vc

하다      3
영향      2
모니터링    2
자금      2
외화      2
외환      2
경제      2
입       1
유출      1
등       1
긴밀하다    1
변화      1
유지      1
안정      1
금융시장    1
국내      1
공조      1
필요      1
있다      1
라며      1
것       1
미치다     1
우리      1
원       1
상황      1
정치      1
환율      1
개월      1
재       1
진입      1
물가      1
금리      1
동결      1
당국      1
긴축      1
장기      1
우려      1
변동성     1
강화      1
국제      1
달러      1
말       1
Name: count, dtype: int64

In [27]:
# 학습한 모델에 있는 형태소 데이터만 추출, 없는 경우 0
temp = pd.DataFrame(columns=X.columns)

for word in vc.index:
    count = vc.loc[word]
    if word in X.columns:
        temp.loc[0, word] = count

temp.fillna(0, inplace=True)

In [28]:
temp

,전투기,외교관,재현,관례,극복,김현주,자행,월요일,기다,실으,...,알고리즘,평점,한층,식품,절규,코레일,숱,강매,시선집중,신의성실
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [29]:
# 예측
sect_dic = {0: "정치", 1: "경제", 2: "사회", 3: "생활/문화", 4: "세계", 5: "IT/과학"}

predict = model.predict(temp)

pred = predict[0]
print("이 기사는 "+sect_dic[pred]+"뉴스입니다")

이 기사는 경제뉴스입니다
